<a href="https://colab.research.google.com/github/yuriao/DataScienceProjects/blob/main/commonlit_bert_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I will explore how to use basic BERT network for scoring, based only on students' response

### References
-

In [18]:
from google.colab import drive
drive.mount('commonLit_data')

Drive already mounted at commonLit_data; to attempt to forcibly remount, call drive.mount("commonLit_data", force_remount=True).


In [19]:
!mkdir working

mkdir: cannot create directory ‘working’: File exists


In [20]:
!pip install transformers[torch]

## Import libraries

In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline, AdamW
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [22]:
import logging
import warnings
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)

## Dataload

In [23]:
DATA_DIR = "/content/commonLit_data/MyDrive/commonLit_data/"

#prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
#prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")


In [24]:
summaries_train.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


## EDA: Check nans

In [25]:
summaries_train.isna().sum()

student_id    0
prompt_id     0
text          0
content       0
wording       0
dtype: int64

In [26]:
summaries_test.isna().sum()

student_id    0
prompt_id     0
text          0
dtype: int64

data is good

## Tokenizer and model

In [27]:
#MODEL_DIR = "kaggle/input/huggingface-bert/"
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)

## encoding training text

In [50]:
encodings = tokenizer(list(summaries_train['text']), return_tensors="pt", padding=True, truncation=True, max_length=32)
test_encodings = tokenizer(list(summaries_test['text']), return_tensors="pt", padding=True, truncation=True, max_length=32)

## Convert scores to tensors

In [36]:
labels = torch.tensor(summaries_train[['content','wording']].values)

## to GPU

In [37]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Dataset class

In [38]:
class RegressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

## fine-tuning metric

In [39]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

## kFold validation setup

In [53]:
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

## kFold training

In [60]:
!pip install -qq wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [68]:
import wandb
wandb.login()


%env WANDB_PROJECT=tuning
%env WANDB_LOG_MODEL=true


env: WANDB_PROJECT=tuning
env: WANDB_LOG_MODEL=true


In [69]:
sweep_config = {
    'method': 'random'
}


# hyperparameters
parameters_dict = {
    'epochs': {
        'value': 1
        },
    'batch_size': {
        'values': [8, 16, 32, 64]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
    },
}

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project='tuning')


Create sweep with ID: 8wqkt85t
Sweep URL: https://wandb.ai/lujiachen1994/tuning/sweeps/8wqkt85t


In [73]:
def train(config=None):
  with wandb.init(config=config):

     # set sweep configuration
    config = wandb.config


    # set training arguments
    training_args = TrainingArguments(
        output_dir='working',
	      report_to='wandb',  # Turn on Weights & Biases logging
        num_train_epochs=config.epochs,
        learning_rate=config.learning_rate,
        weight_decay=config.weight_decay,
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=16,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        logging_strategy='epoch',
        load_best_model_at_end=True,
        remove_unused_columns=False,
        greater_is_better=False,
        fp16=True
    )


    # define training loop
    trainer = Trainer(
        # model,
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )


    # start training loop
    trainer.train()


In [ ]:
#models=[]
for fold, (train_idx, val_idx) in enumerate(kfold.split(summaries_train['text'])):
    print(f"Fold {fold + 1}")

    train_encodings = {key: val[train_idx] for key, val in encodings.items()}
    val_encodings = {key: val[val_idx] for key, val in encodings.items()}
    train_labels = labels[train_idx]
    val_labels = labels[val_idx]

    # Initialize datasets
    train_dataset = RegressionDataset(train_encodings, train_labels)
    val_dataset = RegressionDataset(val_encodings, val_labels)

    # Initialize the model for each fold
    #model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # num_labels=2 for dual regression

    wandb.agent(sweep_id, train, count=100)

## Predict

In [ ]:
with torch.no_grad():
    test_ids = []
    test_attention_mask = []
    test_ids.append(test_encodings['input_ids'])
    test_attention_mask.append(test_encodings['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)
    predictions = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device)).logits

In [ ]:
print(predictions)

## Create Submission file

In [ ]:
summaries_test[["content", "wording"]]=predictions.cpu()
summaries_test[["student_id", "content", "wording"]].to_csv("submission.csv", index=False)